In [8]:
import pandas as pd
import xml.etree.ElementTree as ET
from urllib.request import urlopen
from tqdm import trange
from bs4 import BeautifulSoup as bs
import re

In [2]:
#기존의 크롤링 방ㅇ법 말고 다른 방식을 써보자

id='mugudeli601'
url = "https://www.law.go.kr/DRF/lawSearch.do?OC={}&target=prec&type=XML".format(id)
response = urlopen(url).read()
bs(response)

c:\Users\Heeyo\AppData\Local\Programs\Python\Python39\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


<?xml version="1.0" encoding="UTF-8"?><html><body><precsearch><target>prec</target>&lt;키워드&gt;*키워드&gt;<section>evtNm</section><totalcnt>83460</totalcnt><page>1</page><prec id="1">&lt;판례일련번호&gt;222677판례일련번호&gt;&lt;사건명&gt;&lt;![CDATA[양수금]]&gt;사건명&gt;&lt;사건번호&gt;2021그753사건번호&gt;&lt;선고일자&gt;2022.06.21선고일자&gt;&lt;법원명&gt;대법원법원명&gt;&lt;법원종류코드&gt;법원종류코드&gt;&lt;사건종류명&gt;민사사건종류명&gt;&lt;사건종류코드&gt;400101사건종류코드&gt;&lt;판결유형&gt;결정판결유형&gt;&lt;선고&gt;자선고&gt;&lt;판례상세링크&gt;/DRF/lawService.do?OC=mugudeli601&amp;target=prec&amp;ID=222677&amp;type=HTML&amp;mobileYn=판례상세링크&gt;</prec><prec id="2">&lt;판례일련번호&gt;222699판례일련번호&gt;&lt;사건명&gt;&lt;![CDATA[손해배상(자)]]&gt;사건명&gt;&lt;사건번호&gt;2017다289538사건번호&gt;&lt;선고일자&gt;2022.06.16선고일자&gt;&lt;법원명&gt;대법원법원명&gt;&lt;법원종류코드&gt;법원종류코드&gt;&lt;사건종류명&gt;민사사건종류명&gt;&lt;사건종류코드&gt;400101사건종류코드&gt;&lt;판결유형&gt;판결판결유형&gt;&lt;선고&gt;선고선고&gt;&lt;판례상세링크&gt;/DRF/lawService.do?OC=mugudeli601&amp;target=prec&amp;ID=222699&amp;type=HTML&amp;mobileYn=판례상세링크&gt;</prec><prec id="3">&lt;판례일련번호&gt

In [3]:
xtree = ET.fromstring(response) #response에 저장된 string으로부터 tree를 생성한다

In [27]:
#first check what is in it
'''for child in xtree:
    print(':',child.tag,child.attrib)
print(xtree.find('page').text)'''

1


In [16]:
xtree[3].text #totalCnt

'83460'

In [4]:
for node in xtree[5:]:
    print('\n\n',node.attrib)
    tags=[]
    for child in node:
        tags.append(child.tag)
    for t in tags:
        print(node.find(t).text)



 {'id': '1'}
222677
양수금
2021그753
2022.06.21
대법원
None
민사
400101
결정
자
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=222677&type=HTML&mobileYn=


 {'id': '2'}
222699
손해배상(자)
2017다289538
2022.06.16
대법원
None
민사
400101
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=222699&type=HTML&mobileYn=


 {'id': '3'}
222683
개인정보보호법위반
2022도1676
2022.06.16
대법원
None
형사
400102
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=222683&type=HTML&mobileYn=


 {'id': '4'}
222691
토지인도
2018다301350
2022.06.16
대법원
None
민사
400101
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=222691&type=HTML&mobileYn=


 {'id': '5'}
222679
미성년자약취·사체은닉미수[피고인이 산부인과의원에서 피고인의 딸이 출산한 피해자를 자신이 출산한 이 사건 여아와 바꿔치기 하는 방법으로 피해자를 데리고 가 미성년자를 약취하였다고 기소된 사안]
2022도2236
2022.06.16
대법원
None
형사
400102
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=222679&type=HTML&mobileYn=


 {'id': '6'}
222695
이혼등
2021므14258
2022.06.16
대법원
None
가사
400103
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=222695&type=HTML&mobil

In [6]:
totalCnt = int(xtree.find('totalCnt').text)#판례 페이지에 totalCnt라는 요소가 존재하는 것 같다. 
#왜 20개밖에 없나 했는데 원래 한 페이지당 20개라고 한다. 
#페이지마다 loop을 돌아가야 하는 것

In [10]:
#판례 목록 만들기 
cnt=0
page = 1
rows = []
for i in trange(int(totalCnt / 20)): #기다리는 바를 만드는 tqdm의 range 함수인 trange
    try:
        items = xtree[5:]
    except:
        break
        
    for node in items:
        data={}
        tags=[]
        for child in node:
            tags.append(child.tag)
        for t in tags:
            data[t]=re.sub(',','$',str(node.find(t).text))

        rows.append(data)
    cnt += 1
    url = "https://www.law.go.kr/DRF/lawSearch.do?OC=mugudeli601&target=prec&type=XML&page={}".format(page)
    response = urlopen(url).read()
    xtree = ET.fromstring(response)
    page+=1

  0%|          | 0/4173 [00:00<?, ?it/s]


TypeError: expected string or bytes-like object

In [49]:
#rows에 모든 정보가 담겨있음
prec_list=pd.DataFrame(rows,columns=rows[0].keys())
prec_list.head(5)

,판례일련번호,사건명,사건번호,선고일자,법원명,법원종류코드,사건종류명,사건종류코드,판결유형,선고,판례상세링크
0,221949,임금등,2017다292343,2022.05.26,대법원,None,민사,400101,판결,선고,/DRF/lawService.do?OC=mugudeli601&target=prec&...
1,221985,대여금[지적장애를 가진 사람의 의사능력이 문제된 사건],2019다213344,2022.05.26,대법원,None,민사,400101,판결,선고,/DRF/lawService.do?OC=mugudeli601&target=prec&...
2,221993,토지인도등·손해배상금,2021다216421,2022.05.26,대법원,None,민사,400101,판결,선고,/DRF/lawService.do?OC=mugudeli601&target=prec&...
3,221991,구상금,2020다301186,2022.05.26,대법원,None,민사,400101,판결,선고,/DRF/lawService.do?OC=mugudeli601&target=prec&...
4,221953,상소권회복기각결정에대한재항고,2022모439,2022.05.26,대법원,None,형사,400102,결정,자,/DRF/lawService.do?OC=mugudeli601&target=prec&...


In [55]:
prec_list.columns

Index(['판례일련번호', '사건명', '사건번호', '선고일자', '법원명', '법원종류코드', '사건종류명', '사건종류코드',
       '판결유형', '선고', '판례상세링크'],
      dtype='object')

In [64]:
#prec_list를 traverse하며 모든 판례정보를 긁기
import re

contents = ['판시사항', '판결요지', '참조조문', '참조판례', '판례내용']
case_list=pd.DataFrame(columns=contents)

def remove_tag(content):
    cleaned=re.sub('<.*?>', '', content)
    return cleaned

for pid,l in zip(prec_list['판례일련번호'],prec_list['판례상세링크']):
    url="https://www.law.go.kr"+l.replace('HTML','XML')
    response=urlopen(url).read()
    xtree=ET.fromstring(response)
    row={}
    row['판례일련번호']=pid
    for content in contents:
        text=xtree.find(content).text
        if text is None:
            text="-"
        else:
            text=remove_tag(text)
        row[content]=text
    case_list=pd.concat([case_list,pd.DataFrame([row],columns=row.keys())],axis=0)
case_list.reset_index(inplace=True)
        
        

        

In [65]:
case_list.head(5)

,index,판시사항,판결요지,참조조문,참조판례,판례내용,판례일련번호
0,0,[1] 연령을 이유로 한 차별을 금지하고 있는 구 고용상 연령차별금지 및 고령자...,[1] 구 고용상 연령차별금지 및 고령자고용촉진에 관한 법률(2020. 5. 2...,[1] 구 고용상 연령차별금지 및 고령자고용촉진에 관한 법률(2020. 5. 26...,-,"【원고, 피상고인】 원고 (소송대리인 법무법인(유한) 산경 담당변호사 김선종 외 1...",221949
1,0,[1] 의사능력의 의미 및 의사능력 유무는 구체적인 법률행위와 관련하여 개별적으...,[1] 의사능력이란 자기 행위의 의미나 결과를 정상적인 인식력과 예기력을 바탕으...,"[1] 민법 제9조 제1항, 제12조 제1항, 제14조의2 제1항 / [2] 민법...",[1] 대법원 2002. 10. 11. 선고 2001다10113 판결(공2002하...,"【원고, 피상고인】 비엔케이캐피탈 주식회사【피고, 상고인】 피고(법정대리인 한정후견...",221985
2,0,농지에 관한 임대차계약이 강행법규인 농지법 제23조에 위반되어 무효가 되는 경우...,농지에 관한 임대차계약이 강행법규인 농지법 제23조에 위반되어 무효가 되는 경우...,"농지법 제1조, 제3조, 제23조, 민법 제741조\n","대법원 1994. 6. 28. 선고 93다51539 판결(공1994하, 2089),...","【원고(반소피고), 피상고인】 원고(반소피고) (소송대리인 법무법인 그린 담당변호사...",221993
3,0,[1] 학교안전공제중앙회가 학교배상책임공제에 따라 피해자에게 공제금을 지급한 경...,[1] 학교배상책임공제는 학교안전사고 예방 및 보상에 관한 법률(이하 ‘학교안전...,"[1] 학교안전사고 예방 및 보상에 관한 법률 제15조, 제28조, 상법 제664...",-,"【원고, 피상고인】 학교안전공제중앙회 (소송대리인 법무법인 신의 담당변호사 조주영 ...",221991
4,0,[1] 형사피고사건으로 법원에 재판이 계속 중인 사람은 공소제기 당시의 주소지나...,[1] 형사소송법 제345조의 상소권회복청구는 자기 또는 대리인이 책임질 수 없...,"[1] 형사소송법 제345조 / [2] 형사소송법 제63조 제1항, 제276조, 소...",[1] 대법원 2008. 3. 10. 자 2007모795 결정 / [2] 대법원 ...,【피 고 인】 피고인【재항고인】 피고인【원심결정】 울산지법 2022. 2. 16. ...,221953


In [68]:
from pathlib import Path  
filepath = Path('case_list.csv')  
case_list.to_csv(filepath,index=False)